<p align="center">
<img src="http://wandb.me/logo-im-png" width="800" alt="Weights & Biases" />
</p>

# 🪐 Operation REBOOT: Mission Start

Welcome, **Neural Architect**. The ship's AI core is down. Your job: fine-tune a foundational model with astrological Q&A data to restore its deep space reasoning abilities.

**Your mission:**
- Adjust dataset splits
- Configure training arguments
- Launch training and monitor with **Weights & Biases (W&B)**
- Test and evaluate your fine-tuned model

All systems go. Let's bring this vessel back online.

#### Install and Import  

In [1]:
!pip install transformers datasets accelerate wandb trl huggingface_hub bitsandbytes peft torchvision -q

In [2]:
import wandb

import json
import math
import random
from pathlib import Path


import torch
import torchvision
import pandas as pd
from datasets import Dataset
from transformers import TrainingArguments, Trainer

In [3]:
from utilities.helpers import *

## 🔌 Connect Neural Telemetry (W&B Setup)

In [4]:
# When prompted to authorize your
# wandb: You can find your API key in your browser here: https://wandb.ai/authorize
# wandb: Paste an API key from your profile and hit enter:

#wandb.login(key=wandb_key)
wandb.login()

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anushrav-vatsa (wandb) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
WANDB_PROJECT_NAME = "Astros-FT-Workshop"
WANDB_ENTITY = "FT-Testing" #Set your W&B Entity #TODO - Change to None for final release
WORKSHOP_TEAM_NAME = "Wandb_Crew" #Set to your Workshop Team Name #TODO - Change to None for final release

## 🧪 Dataset Control Room
Adjust the dataset splits and prepare the astrological QA dataset for training.

## 🌎 Initialize Experiment, Read Data, Split Data ☄️

In this section, we:

* Retrieve the Astros Dataset from [W&B Registry FC_FT_Workshop_Dataset collection](https://wandb.ai/orgs/FullyConnected-2025-Workshops/registry/dataset?selectionPath=fullyconnected-2025-workshops%2Fwandb-registry-dataset%2FFC_FT_Workshop_Dataset&view=versions) TODO: Update for final version
* Load the Astros Dataset containing universe-related Q&A data.
* Create prompts from the question/answer pairs & load into a pandas dataframe
* Convert the pandas DataFrame into a Hugging Face Dataset.

✅ All the heavy lifting is done here automatically — no manual setup needed

#### Let's prepare our training dataset

In [6]:
# Step 1: Initialize W&B run and download dataset
print("Step 1: Downloading dataset from Weights & Biases...")
run = wandb.init(entity=WANDB_ENTITY,
                 project=WANDB_PROJECT_NAME,
                 job_type="data_retrieval",
                 name="fetch_astros_dataset"
                 )

# Download the dataset artifact
artifact = run.use_artifact('wandb-registry-dataset/FC_FT_Workshop_Dataset:v4', type='dataset')
dataset_dir = artifact.download()
run.finish()
print("✅ Dataset downloaded successfully!")

# Step 2: Load and prepare datasets
df_train, training_dataset = load_and_prepare_dataset(dataset_dir, "astro_dataset_train.jsonl", "training") #look at the helper fuctions if you're interested in how we prepare the data

# Print dataset statistics
print("\nDataset Statistics:")
print(f"Training examples: {len(df_train)}")
print("\nExample prompt format:")
print(df_train['text'].iloc[0])

Step 1: Downloading dataset from Weights & Biases...


wandb:   1 of 1 files downloaded.  


✅ Dataset downloaded successfully!

Loading training dataset...
✅ Successfully loaded dataset with no errors.
✅ Training dataset loaded with 1600 examples

Dataset Statistics:
Training examples: 1600

Example prompt format:
Question: What are 'Superluminous Supernovae' (SLSNe) and what distinguishes Type I SLSNe from normal Type Ia supernovae spectroscopically?
Answer: Superluminous Supernovae (SLSNe) are much more luminous than normal Type Ia supernovae. Spectroscopically, Type I SLSNe are characterized by the absence of hydrogen and strong helium lines near peak light (like normal SNe Ia), but they show strong, broad metal lines, often including oxygen, magnesium, and calcium. Normal SNe Ia are defined by the presence of strong silicon absorption lines (Si II λ6355) near peak light, which are often weak or absent in SLSNe I. The differences in spectra indicate different progenitor systems and explosion mechanisms: SNe Ia are thermonuclear disruptions of white dwarfs, while SLSNe I ar

### 🌌 Dataset Loaded Successfully!

At this point, we've:
* Retrieved the Astros Dataset artifact
* Loaded it into a pandas DataFrame
* Created prompt-style text for fine-tuning
* Converted it into a Hugging Face Dataset for training

✨ Feel free to pause and explore the data before moving forward!

Exploring the dataset can help you:

* Understand the kinds of questions and answers the model will learn from
* Check for any strange patterns, formatting issues, or interesting insights
* Discover Easter Eggs

🛡️ We've added soft error handling while loading, so if you accidentally modify the dataset file, you'll be warned if any loading issues happen.

👉 Quick Tip: You don't need to modify the dataset to proceed, but if you want to explore, you can run things like:

```
print(df_train.sample(5))
print(df_train['question'].apply(len).describe())
print(df_train['answer'].apply(len).describe())
```

When you're ready, move on to loading the model and tokenizing the dataset!

## 🧠 Model Vault: Load & Configure the Neural Core

## 🚀 Load Pretrained Model and Prepare Dataset for Fine-Tuning 🌠
In this section, we:

* Retrieve the Model to Finetune from [W&B Registry FC_FT_Workshop_Model](https://wandb.ai/orgs/FullyConnected-2025-Workshops/registry/model?selectionPath=fullyconnected-2025-workshops%2Fwandb-registry-model%2FFC_FT_Workshop_Model&view=versions) TODO: Update for final version
* Select and load a pretrained language model and its tokenizer from Hugging Face.
* Format the Astros prompts into tokenized input IDs the model can understand.
* Apply padding and truncation to keep sequence lengths manageable.
* Split the tokenized dataset into training and validation sets (90% train / 10% validation).
* Define a compute_metrics function to track TODO during fine-tuning.

✅ All the setup for model loading, tokenization, and data splitting is handled for you — no manual steps required!

### Select Model
You will be prompted to select one of the following models

*   Option 1: [falcon-rw-1b](https://huggingface.co/tiiuae/falcon-rw-1b)
*   Option 2: [TinyLlama](https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0)


In [7]:
model_name, version = get_model_from_wandb(WANDB_ENTITY, WANDB_PROJECT_NAME)


Available Models:
1. Falcon RW 1B
2. TinyLlama 1B



Select a model (1-2):  1



✅ Selected: falcon-rw-1b

⬇️ Downloading falcon-rw-1b (version v0) from Weights & Biases...


wandb: Downloading large artifact FC_FT_Workshop_Models:v0, 2505.10MB. 12 files... 
wandb:   12 of 12 files downloaded.  
Done. 0:0:2.2 (1125.7MB/s)


✅ Model saved to: models/falcon-rw-1b_v0
✅ Model downloaded successfully!




Next, we'll make a few adjustments to ensure the model handles padding correctly,
and then prepare our dataset for training by tokenizing the input prompts.


## 🔄 Tokenize & Split: Format Data for Finetuning

#### Load the datasets

You can modify how our training data is passed to our training script to finetune the model. Make sure to analyze the data so you can select an appropriate **Sample Size** and  **Train/Test split** for the finetuning process.

TO-DO: Update the values below for your first run. You can come back and try it with new values again.

In [8]:
training_sample = training_dataset.shuffle(seed=42).select(range(500)) # chosse between 100 and 1600 samples

train_test_split = 0.1 # choose a float value between 0 and 1

## ⚙️ Training Command Center
Set training arguments to guide your model's learning trajectory.

## 🛰️ Training Arguments (Where You Fine-Tune Settings) 🌙

This is where you'll do most of your experimentation! 🎯

The `TrainingArguments` object controls how your model is fine-tuned, including:

* Batch size
* Number of epochs
* Learning rate
* Warmup steps
* Mixed precision (fp16) for faster training
* Checkpoint saving
* Reporting to Weights & Biases

You can modify the hyperparameters here to see how different settings impact model performance.

🔥 Pro Tip: TODO INSERT SOME TIPS


In [9]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    run_name=f"fine-tuning-{model_name}-qlora",
    output_dir="./results",
    num_train_epochs=2, #start with 3 and go upto 10 epochs for better results
    per_device_train_batch_size=32,
    per_device_eval_batch_size=4,
    dataloader_num_workers=16,
    gradient_accumulation_steps=1,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    do_train=True,
    do_eval=True,
    fp16=True,
    bf16=False,
    gradient_checkpointing=True, #Choose to store the full forward-pass activations in GPU RAM
    group_by_length=True,
    report_to=["wandb"],
    remove_unused_columns=True,
    dataloader_pin_memory=True,
    optim="adamw_torch", #See https://huggingface.co/docs/transformers/v4.51.3/en/perf_train_gpu_one#optimizers
    learning_rate=2e-3,
    lr_scheduler_type="cosine", # See https://huggingface.co/docs/transformers/en/main_classes/optimizer_schedules#transformers.SchedulerType
    auto_find_batch_size=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=5,
    load_best_model_at_end=False,
    metric_for_best_model="eval_loss",
    logging_strategy="steps",
    label_names=["labels"],
)

## 🛰️ Engage Training Tracker
Launch the model and track training live with W&B.

## 🔭 Initialize Trainer, Train, and Save 🌎

In this final section:

* We initialize the Trainer with:
  * The model
  * The datasets (train/test splits)
  * The training arguments
  * A data collator for language modeling
  * Our compute_metrics function to calculate TODO

* We start training by calling trainer.train().
* We save the fine-tuned model and tokenizer locally.
* We finish the W&B run to close the logging cleanly.

🧠 Reminder: After training finishes, your fine-tuned model will be available in your local runtime — you can upload it back to W&B or Hugging Face later!

🚨 Training Ahead: Be ready for 10-15 min runtimes with the default configs!


Running into the following?

> AttributeError: `AcceleratorState` object has no attribute `distributed_type`

This happens if `AcceleratorState._reset_state()` was called and an `Accelerator` or `PartialState` was not reinitialized.

To Fix Run the following in a new cell before reinitalizing your trainer

```
from accelerate.state import AcceleratorState
AcceleratorState._reset_state()
```




In [10]:
model, tokenizer, model_name = load_model_and_tokenizer(model_name, version)
train_dataset, eval_dataset = tokenized_train_test(training_dataset, train_test_split, tokenizer)


📦 Loading model from: models/falcon-rw-1b_v0

Step 1: Loading tokenizer...
✅ Tokenizer loaded successfully!

Step 2: Configuring QLoRA...
Loading model with 4-bit quantization...


You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


trainable params: 12,582,912 || all params: 1,324,208,128 || trainable%: 0.9502
✅ Model loaded with QLoRA successfully!


Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

✅ Tokenization applied to Training & Evaluation Datasets successfully!


In [12]:

#Configure model for training
model.config.use_cache = False  # Disable cache during training

# Set label names for PEFT model
model.config.label_names = ["labels"]

# Initialize trainer with modified configuration
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
        pad_to_multiple_of=8  # Add padding to multiple of 8 for better performance
    ),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    #compute_metrics=compute_perpexity, #If monitoring additional metric(s) can set this argument accordingly to your compute functions
)

# Enable gradient checkpointing with the new format
if hasattr(model, "enable_input_require_grads"):
    model.enable_input_require_grads()

## ⚙ Now we kick off the training process ⚙

In [ ]:
#Train
run = wandb.init(entity=WANDB_ENTITY,
                  project=WANDB_PROJECT_NAME,
                  job_type="finetuning_job",
                  name = f"fine_tune_{model_name}"
                  )
train_output = trainer.train()

wandb.finish()

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,2.265400,2.090923


## 💾 Save & Upload
Preserve your fine-tuned model as a W&B artifact.

We will now save this model to W&B. You will need the artifact refernce for this model to get points for completing the fine tuning section.

Tracking your model in W&B can be really helpful:

- You can now share this model with your team and beyond
- W&B creates a lineage map of your model so you can see the full model lifecycle dataset->training->final state

In [14]:
#Saving and uploading best model - working draft
trainer.save_model(f"./best_model/{type(model.base_model.model).__name__}")
tokenizer.save_pretrained(f"./best_model/{type(model.base_model.model).__name__}")

run = wandb.init(project=WANDB_PROJECT_NAME,
                 entity=WANDB_ENTITY,
                 job_type="FT-Workshop-Finetuning-Best-Model-Upload",
                 name="FT-Best-Model-Upload")

artifact = wandb.Artifact(
    name=f"{WORKSHOP_TEAM_NAME}-ft-best-model-{type(model.base_model.model).__name__}",
    type="model",
    description="""Best FineTuned model from the Astros-FT-Workshop."""
)

artifact.add_dir(local_path="./best_model")

logged_artifact = run.log_artifact(artifact)

run.link_artifact(
  artifact=logged_artifact,
  target_path="wandb-registry-model/FC_FT_Workshop_FineTuned_Models"
)

run.finish()

wandb: Adding directory to artifact (./best_model)... Done. 0.1s


## ✅ Mission Checkpoint: Model Finetuned

Congratulations, Architect! You've:
- Loaded and prepped your training dataset ✅
- Configured a foundational model ✅
- Finetuned it with parameter-efficient methods ✅
- Logged your training runs and saved the final model to Weights & Biases ✅

Your model is now part of your mission's neural infrastructure.

Next, we prepare to test and evaluate. But first, a quick system check...

## 🧰 Systems Maintenance Bay: Utilities

Before testing, it's wise to flush memory and check your hardware status. Use these utilities to prepare the environment.

Just like a good engineer, make sure the ship's neural bays are cleared and ready.

## Utilities 🧰

In [15]:
# -- Flush out GPU memory - when required - may require restarting the notebook
import gc, torch

try:
    del trainer
except: print("cannot release memory")
try:
    del model
except: print("cannot release memory")
try:
    del tokenizer
except: print("cannot release memory")

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

wandb.finish()

In [16]:
!nvidia-smi

Sun May 25 00:17:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      On  |   00000000:36:00.0 Off |                    0 |
| N/A   48C    P0             30W /   72W |    1599MiB /  23034MiB |    100%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 🧪 Testing the Neural Core

Now that your model is trained and uploaded, it’s time to test your ship’s new neural core.

You’ll load the fine-tuned model and run test prompts to ensure it responds with precision and depth—critical for deep-space operations.

We’ve equipped you with a call function wrapped in `Weave`, our GenAI interface and telemetry layer.

# 🔧 Testing our model 🪛

Let's start by creating some helper functions to load and call the model we just trained.

Since we created an adapter during the finetuning process, our load model function loads the original model along with our adapter using PEFT.

## 🛰️ Introducing Weave: Your AI Telemetry and Evaluation Suite

**Weave** is Weights & Biases’ next-gen platform for tracking, evaluating, and visualizing GenAI applications.

In REBOOT, you'll use Weave to:
- Log and score model generations
- Run structured evaluations on Q&A performance
- Compare outputs with reference answers

This enables you to **quantitatively assess** how mission-ready your model is.

Let’s initialize Weave and plug it into your finetuned system.

In [17]:
!pip install weave "weave[scorers]" -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.243.3 requires numpy==1.26.4, but you have numpy 2.2.6 which is incompatible.


In [18]:
import weave
weave.init(f"{WANDB_ENTITY}/{WANDB_PROJECT_NAME}")

weave: Logged in as Weights & Biases user: anushrav-vatsa.
weave: View Weave data at https://wandb.ai/FT-Testing/Astros-FT-Workshop/weave


### Calling our Local Finetuned Model

In [19]:
@weave.op()
def call_model(question: str) -> str:
    """Generate an answer from your Local LLM given a prompt."""

    system_prompt = "You are an expert in astrophysics. Please provide a concise and truthful answer to the following question:"
    prompt = system_prompt + "\n\n" + question + "\nAnswer:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=150, do_sample=False, eos_token_id=tokenizer.eos_token_id, pad_token_id=model.config.eos_token_id)
    return tokenizer.decode(output[0], skip_special_tokens=True).replace(prompt, '').strip(),

## 📊 Final Check: Evaluation Protocols

Your neural core is active—but is it mission-grade?

Use this section to:
- Load an evaluation dataset
- Score model responses using embedding similarity
- Track performance with W&B + Weave

**Evaluation is critical** before deployment—it ensures your model’s reasoning is aligned with mission parameters.

# Evaluating our Model

## Load model

In [20]:
base_model_dir = "./models/falcon-rw-1b_v0" # Path to base model - modify accordingly to fine_tuned_model/<TinyLlama_v1 or falcon-rw-1b_v0>
adapter_dir = "./best_model/FalconForCausalLM" #add path to adapter dir

tokenizer, model = load_finetuned_model(adapter_dir, base_model_dir)

Loading model with 4-bit quantization...
✅ Loaded model with finetuned adapter successfully!


## Get Eval Dataset

In [21]:
weave.init('fc25-wandb-admins/uncategorized')
eval_dataset_public_v0 = weave.ref('eval_dataset_public:v0').get()

weave: Logged in as Weights & Biases user: anushrav-vatsa.
weave: View Weave data at https://wandb.ai/fc25-wandb-admins/uncategorized/weave
weave: retry_attempt


## Test the model with a sample from our eval dataset

In [22]:
question = eval_dataset_public_v0[10]['question']
answer = call_model(question)

print("🛰️  Incoming Transmission — Mission Q&A\n")
print(f"🧠 Question:\n{question}\n")
print(f"🤖 Model Response:\n{answer}")

weave: 🍩 https://wandb.ai/fc25-wandb-admins/uncategorized/r/call/019704ce-f4c3-7e22-b42a-cbade1c8b293


🛰️  Incoming Transmission — Mission Q&A

🧠 Question:
What is the 'photon sphere' around a black hole and how is it related to the black hole shadow?

🤖 Model Response:
("The photon sphere is a hypothetical region of the universe around a black hole where photons are accelerated to high energies by gravitational potentials. The photon sphere is predicted to be a sphere of radius roughly equal to the black hole radius, surrounded by a halo of photons. The halo is predicted to be filled with photons accelerated by the black hole's strong gravitational potential, similar to the accretion disk around a supermassive black hole. The photon sphere is thought to be a potential site for gravitational waves from the accretion onto the black hole, potentially providing a probe of the event horizon and the strong gravity of the black hole.</s>s>\nAnswer: The photon sphere is a hypothetical region of the universe around a black hole where photons are accelerated",)


weave: 🍩 https://wandb.ai/fc25-wandb-admins/uncategorized/r/call/019704d0-e86a-7290-98ec-f77f080ddbce
weave: 🍩 https://wandb.ai/fc25-wandb-admins/uncategorized/r/call/019704d2-77d4-73b2-8328-75ae85206a4f
weave: 🍩 https://wandb.ai/fc25-wandb-admins/uncategorized/r/call/019704d4-9f23-7442-bbb2-2321b291ceb2
weave: 🍩 https://wandb.ai/fc25-wandb-admins/uncategorized/r/call/019704db-9808-7d80-8a1c-0d3354b3a94e
weave: 🍩 https://wandb.ai/fc25-wandb-admins/uncategorized/r/call/019704db-d775-77f2-826e-e6cfb05acddc


## Setup eval

In [30]:
import asyncio
from weave.scorers import EmbeddingSimilarityScorer
similarity_scorer = EmbeddingSimilarityScorer(
    model_id="bedrock/amazon.titan-embed-text-v2:0",  # will need to update this to bedrock's titan models
    threshold=0.7
)

similarity_scorer.column_map = {
    "output": "model_output",  # Your model's response
    "target": "answer", # The reference response (expected)
    "kwargs": "question"
}

In [34]:
eval_dataset = eval_dataset_public_v0.rows[20:21] # select samples to run evalaution against

In [35]:
evaluation = weave.Evaluation(
    evaluation_name = f"{WORKSHOP_TEAM_NAME}_qna_eval",
    dataset=eval_dataset, scorers=[similarity_scorer],
    name="model_qna_eval"
   )

print(await(evaluation.evaluate(call_model)) )

weave: Predict and score failed
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/litellm/llms/bedrock/embed/embedding.py", line 138, in _make_async_call
    response = await client.post(url=api_base, headers=headers, data=json.dumps(data))  # type: ignore
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/litellm/litellm_core_utils/logging_utils.py", line 135, in async_wrapper
    result = await func(*args, **kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/litellm/llms/custom_httpx/http_handler.py", line 276, in post
    raise e
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/litellm/llms/custom_httpx/http_handler.py", line 232, in post
    response.raise_for_status()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, 


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

{'EmbeddingSimilarityScorer': None}
